In [5]:
# import modules
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge

In [3]:
# download data
# DOWNLOADING THE DATASET: UNCOMMENT BELOW:
# !wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-01.parquet
# !wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet

--2022-07-14 17:22:53--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-01.parquet
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.216.129.123
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.216.129.123|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2022-07-14 17:22:54 ERROR 403: Forbidden.

--2022-07-14 17:22:54--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.216.129.123
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.216.129.123|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2022-07-14 17:22:55 ERROR 403: Forbidden.



In [7]:
df = pd.read_parquet('./fhv_tripdata_2021-01.parquet"')

OSError: [Errno 22] Invalid argument: './fhv_tripdata_2021-01.parquet"'

In [ ]:
df.head()

In [8]:
len(df)

NameError: name 'df' is not defined

Q. Read the data for January. How many records are there?
        
        Ans: 1154112


### Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.



In [ ]:
df["duration"] = df["lpep_dropoff_datetime"] - df["lpep_pickup_datetime"]
df["duration"] = df["duration"].apply(lambda x: x.total_seconds() / 60)


In [ ]:
df.duration.mean()

What's the average trip duration in January?

        Ans: 19.16

### Data preparation
Check the distribution of the duration variable. There are some outliers.

Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

How many records did you drop?



In [9]:
print("Before: ", len(df))
prev = df
df = df[(df.duration >= 1) & (df.duration <=60)]
print("After: ", len(df))
print("No. of records were droped:", len(prev) - len(df))

NameError: name 'df' is not defined

### Q3. Missing values
The features we'll use for our model are the pickup and dropoff location IDs.

But they have a lot of missing values there. Let's replace them with "-1".

What's the fractions of missing values for the pickup location ID? I.e. fraction of "-1"s after you filled the NAs.

In [ ]:
df.isnull().mean()

        ANS: 83%

In [ ]:
categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')

In [ ]:
df.dtypes

### Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

1. Turn the dataframe into a list of dictionaries
3. Fit a dictionary vectorizer
4. Get a feature matrix from it
    
    What's the dimensionality of this matrix? (The number of columns).

In [ ]:
# Initializing one hot encoding:
train_dicts = df[categorical].to_dict(orient="records") # initialize dict
dict_v = DictVectorizer()
X_train = dict_v.fit_transform(train_dicts)
y_train = df['duartion'].values
print(X_train.shape)

    Ans: 525

### Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

* Train a plain linear regression model with default parameters
* Calculate the RMSE of the model on the training data

What's the RMSE on train?



In [ ]:
# Training Linear Regression
lr = LinearRegression()                                        # Intialization
lr.fit(X_train, y_train)                                       # Training model
y_pred = lr.predict(X_train)                                   # Initializing prediction
rmse_error = mean_squared_error(y_train, y_pred, squared=False) # calculating mse
print(rmse_error)

In [ ]:
# Visualization of prediction
sns.distplot(y_train, label="actual")
sns.distplot(y_pred, label="prediction")
plt.legend()

    Ans: 10.52

### Q6. Evaluating the model
Now let's apply this model to the validation dataset (Feb 2021).

What's the RMSE on validation?

In [10]:
def read_dataframe(filename):
    if filename.endswith(".csv"):
        df = pd.read_csv(filename)
        df["lpep_dropoff_datetime"] =  pd.to_datetime(df["lpep_dropoff_datetime"])
        df["lpep_pickup_datetime"] = pd.to_datetime(df["lpep_pickup_datetime"])
    elif filename.endswith(".parquet"):
        df = pd.read_parquet(filename)
    else:
        print("Only .csv | parquet files are allowed")
        return
    df["duration"] = df["lpep_dropoff_datetime"] - df["lpep_pickup_datetime"]
    df["duration"] = df.duration.apply(lambda x: x.total_seconds() / 60)    # convert into minutes
    df = df[(df.duration >=1) & (df.duration <= 60)]
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].fillna(-1).astype('int').astype(str)
    return df

In [ ]:
# Initalizing train and validation DataFrame:
df_train = read_dataframe("./fhv_tripdata_2021-01.parquet")
df_val = read_dataframe("./fhv_tripdata_2021-02.parquet")
print(df_train.shape)
print(df_val.shape)

In [ ]:
# Initialize one hot encoding in a function:
def one_hot_encoding(df_train, df_val, categorical):
    '''
    categorical: List of categorical col names
    numerical: List of numerical col names
    '''
    dict_v = DictVectorizer()
    train_dicts = df_train[categorical].to_dict(orient="records") # initialize dict
    X_train = dict_v.fit_transform(train_dicts)
    val_dicts = df_val[categorical ].to_dict(orient="records")
    X_val = dict_v.transform(val_dicts)
    target = "duration"
    y_train = df_train[target].values
    y_val = df_val[target].values
    return dict_v, X_train, X_val, y_train, y_val

In [ ]:
categorical = ["PU_DO"]
dv, X_train, X_val, y_train, y_val = one_hot_encoding(df_train, df_val, categorical)

In [ ]:
# Training Linear Regression
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_val)
print(mean_squared_error(y_val, y_pred, squared=False)) # print RMSE

    Ans: 11.02

In [ ]:
# Visualization of prediction
sns.distplot(y_val, label="actual")
sns.distplot(y_pred, label="prediction")
plt.legend()